In [1]:
import os
import dask
import json
from os.path import join, dirname, expanduser, splitext, basename
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import leafmap.leafmap as leafmap
from dotenv import load_dotenv
import xarray as xr
import rioxarray as rxr
from PIL import Image
from shapely.geometry import mapping, Polygon
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

# import delayed from dask
import dask

load_dotenv()

True

In [2]:
state="bharatpur"
save_region="bharatpur"
base_path="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns"
mosaic_id="global_quarterly_2024q1_mosaic"
image_meta_data=f"{base_path}/raw_data/metadata/{state}/{mosaic_id}/metadata.geojson"
imagery_dir = f"{base_path}/raw_data/imagery/{mosaic_id}"
state_label_path=f"{base_path}/final_data/labels/{state}.geojson"
stete_shape_path=f"{base_path}/regions/shapes/{state}.geojson"
save_dir=f"../data/region_performance/{state}/"

print(f"state: {state}")
print(f"base_path: {base_path}")
print(f"mosaic_id: {mosaic_id}")
print(f"image_meta_data: {image_meta_data}")
print(f"imagery_dir: {imagery_dir}")
print(f"state_label_path: {state_label_path}")
print(f"stete_shape_path: {stete_shape_path}")
print(f"save_dir: {save_dir}")


state: bharatpur
base_path: /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns
mosaic_id: global_quarterly_2024q1_mosaic
image_meta_data: /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/raw_data/metadata/bharatpur/global_quarterly_2024q1_mosaic/metadata.geojson
imagery_dir: /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/raw_data/imagery/global_quarterly_2024q1_mosaic
state_label_path: /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/final_data/labels/bharatpur.geojson
stete_shape_path: /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/regions/shapes/bharatpur.geojson
save_dir: ../data/region_performance/bharatpur/


In [3]:
# gdf_labels = gpd.read_file(state_label_path).drop("style", errors="ignore", axis=1)
# print(gdf_labels.head())
color_mapping = {"CFCBK": "red", "FCBK": "orange", "Zigzag": "green"}
# gdf_labels["style"] = gdf_labels["class_name"].apply(lambda x: {"color": color_mapping[x]})
# print(gdf_labels.head())
gdf_images = gpd.read_file(image_meta_data)
# print(gdf_images.head())
# print(len(gdf_labels), len(gdf_images))
gdf_image_paths = [join(f"{base_path}/raw_data/imagery/global_quarterly_2024q1_mosaic", f"{Id}.tif") for Id in gdf_images["id"]]
print(len(gdf_image_paths))
print(gdf_image_paths[:5])



24
['/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/raw_data/imagery/global_quarterly_2024q1_mosaic/1461-1188.tif', '/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/raw_data/imagery/global_quarterly_2024q1_mosaic/1462-1188.tif', '/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/raw_data/imagery/global_quarterly_2024q1_mosaic/1463-1188.tif', '/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/raw_data/imagery/global_quarterly_2024q1_mosaic/1461-1187.tif', '/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/raw_data/imagery/global_quarterly_2024q1_mosaic/1462-1187.tif']


In [4]:
m=leafmap.Map()
m.add_basemap("HYBRID")
m.add_geojson(image_meta_data,zoom_to_layer=True)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [5]:
size_in_gb = sum([os.path.getsize(fp)/1024/1024/1024 for fp in gdf_image_paths])
print(f"Total size of images: {size_in_gb:.2f} GB")

Total size of images: 1.18 GB


In [6]:
print(len(gdf_image_paths))
x = np.unique(np.concatenate([xr.open_dataset(path).x.values for path in gdf_image_paths]))
x.sort()
xdiff_index = pd.Series(x).diff().value_counts().index
print(xdiff_index)
print(x)
print(len(x),len(x)/4096)
print(len(gdf_image_paths))
y = np.unique(np.concatenate([xr.open_dataset(path).y.values for path in gdf_image_paths]))
y.sort()
diff_index = pd.Series(y).diff().value_counts().index
print(diff_index)
print(len(y), len(y)/4096)

24
Index([4.777314268052578, 4.777314266189933, 4.7773139998316765,
       4.777315000072122],
      dtype='float64')
[8551165.615795   8551170.39310927 8551175.17042353 ... 8648990.68004347
 8648995.45735773 8649000.234672  ]
20480 5.0
24
Index([ 4.777314267121255,  4.777314267586917, 4.7773139998316765,
        4.777315000072122,  4.777314000297338],
      dtype='float64')
28672 7.0


In [7]:
ds=xr.open_mfdataset(gdf_image_paths)
print(ds)
assert len(ds.x.values) % 4096 == 0
assert len(ds.y.values) % 4096 == 0

<xarray.Dataset> Size: 9GB
Dimensions:      (band: 4, y: 28672, x: 20480)
Coordinates:
  * band         (band) int64 32B 1 2 3 4
  * x            (x) float64 164kB 8.551e+06 8.551e+06 ... 8.649e+06 8.649e+06
  * y            (y) float64 229kB 3.229e+06 3.229e+06 ... 3.092e+06 3.092e+06
    spatial_ref  int64 8B 0
Data variables:
    band_data    (band, y, x) float32 9GB dask.array<chunksize=(1, 512, 512), meta=np.ndarray>


In [8]:
image_size=640
overlap=64
gap_between_centers=image_size-overlap
x_centers=[image_size//2]
x_centers.extend(list(range(x_centers[0]+gap_between_centers, len(ds.x), gap_between_centers)))
x_centers=x_centers[:-1]
y_centers=[image_size//2]
y_centers.extend(list(range(y_centers[0]+gap_between_centers, len(ds.y), gap_between_centers)))
y_centers=y_centers[:-1]
print(len(x_centers), len(y_centers))

34 49


In [9]:
x_values=ds.x.values
y_values=ds.y.values

def get_geometry(x_idx,y_idx):
    start_x=x_values[x_idx-image_size//2] #left boundary
    start_y=y_values[y_idx-image_size//2] #buttom boundary
    end_x=x_values[x_idx+image_size//2] #right boundary
    end_y=y_values[y_idx+image_size//2] #top boundary
    return Polygon([(start_x,start_y),(end_x,start_y),(end_x,end_y),(start_x,end_y)])

print(get_geometry(x_centers[0],y_centers[0]))

X,Y=np.meshgrid(x_centers,y_centers)
geometries=[get_geometry(x,y) for x,y in tqdm(zip(X.ravel(),Y.ravel()))]
x_indices=[x for x in X.ravel()]
y_indices=[y for y in Y.ravel()]
print(len(geometries),len(x_indices),len(y_indices))


POLYGON ((8551165.615795 3228697.685667, 8554223.09692598 3228697.685667, 8554223.09692598 3225640.2045360208, 8551165.615795 3225640.2045360208, 8551165.615795 3228697.685667))


0it [00:00, ?it/s]

1666 1666 1666


In [33]:
# x_indices
# y_indices

In [10]:
potential_image_gdf=gpd.GeoDataFrame(geometry=geometries)
print(len(potential_image_gdf))
potential_image_gdf["x_idx"]=x_indices
potential_image_gdf["y_idx"]=y_indices
potential_image_gdf.reset_index(inplace=True,drop=True)
print(ds.rio.crs)   
potential_image_gdf.crs=ds.rio.crs
print("number of potential images: ",len(potential_image_gdf))
potential_image_gdf.head(2)


1666
EPSG:3857
number of potential images:  1666


,geometry,x_idx,y_idx
0,"POLYGON ((8551165.616 3228697.686, 8554223.097...",320,320
1,"POLYGON ((8553917.349 3228697.686, 8556974.83 ...",896,320


In [11]:
shape_gdf=gpd.read_file(stete_shape_path)
print(shape_gdf.crs)
shape_gdf=shape_gdf.to_crs("EPSG:3857")
assert potential_image_gdf.crs == shape_gdf.crs
print(shape_gdf.crs)
images_within_shape = gpd.sjoin(shape_gdf, potential_image_gdf, predicate="contains")
images_within_shape = potential_image_gdf.loc[images_within_shape.index_right]
display(images_within_shape.head(2))
len(images_within_shape)



EPSG:4326
EPSG:3857


,geometry,x_idx,y_idx
1438,"POLYGON ((8578682.946 3113124.899, 8581740.427...",6080,24512
1474,"POLYGON ((8584186.412 3110373.166, 8587243.893...",7232,25088


686

In [12]:
m = leafmap.Map()
m.add_basemap("HYBRID")
m.add_gdf(shape_gdf, layer_name="Shape", style={"color": "black"})
m.add_gdf(images_within_shape, layer_name="Images within shape",zoom_to_layer=True)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [13]:
images_within_shape['x'] = images_within_shape['x_idx'].apply(lambda x: int(x_values[x]))
images_within_shape['y'] = images_within_shape['y_idx'].apply(lambda x: int(y_values[x]))
images_within_shape.head(6)

,geometry,x_idx,y_idx,x,y
1438,"POLYGON ((8578682.946 3113124.899, 8581740.427...",6080,24512,8580211,3111596
1474,"POLYGON ((8584186.412 3110373.166, 8587243.893...",7232,25088,8585715,3108844
1440,"POLYGON ((8584186.412 3113124.899, 8587243.893...",7232,24512,8585715,3111596
1439,"POLYGON ((8581434.679 3113124.899, 8584492.16 ...",6656,24512,8582963,3111596
1405,"POLYGON ((8581434.679 3115876.632, 8584492.16 ...",6656,23936,8582963,3114347
1404,"POLYGON ((8578682.946 3115876.632, 8581740.427...",6080,23936,8580211,3114347


In [38]:
os.makedirs(f"../data/{save_region}", exist_ok=True)
images_within_shape.reset_index(drop=True).to_file(f"../data/{save_region}/metadata.geojson", driver="GeoJSON")

In [39]:
samples = []
x_idx_list = []
y_idx_list = []
for x_idx, y_idx in tqdm(zip(images_within_shape["x_idx"], images_within_shape["y_idx"])):
    x = x_values[x_idx]
    y = y_values[y_idx]
    save_path = join(save_dir, "images", f"{int(x)}_{int(y)}.tif")
    if os.path.exists(save_path):
        try:
            image = Image.open(save_path)
            assert image.size == (image_size, image_size)
            continue
        except Exception as e:
            print(f"Error: {e}")
            print(f"Loading {save_path} failed. Overwriting the file.")
    sample = ds.isel(x=slice(x_idx - image_size//2, x_idx + image_size//2), y=slice(y_idx - image_size//2, y_idx + image_size//2), band=slice(0, 3))
    samples.append(sample)
    x_idx_list.append(x_idx)
    y_idx_list.append(y_idx)

0it [00:00, ?it/s]

In [40]:
print(len(samples), len(x_idx_list), len(y_idx_list))

686 686 686


In [41]:
def save_sample(sample, x_idx, y_idx):
    x = x_values[x_idx]
    y = y_values[y_idx]
    save_path = join(save_dir, "images", f"{int(x)}_{int(y)}.tif")
    
    sample = sample.band_data.load()
    # fill NaN values with 0
    sample = sample.fillna(0)
    # convert to uint8
    sample = sample.astype(np.uint8)
    sample.rio.to_raster(save_path)
    del sample

In [42]:
delayed_tasks = [dask.delayed(save_sample)(sample, x_idx, y_idx) for sample, x_idx, y_idx in tqdm(zip(samples, x_idx_list, y_idx_list))]
print("Number of delayed tasks:", len(delayed_tasks))

0it [00:00, ?it/s]

Number of delayed tasks: 686


In [43]:
os.makedirs(join(save_dir, "images"), exist_ok=True)
for i in tqdm(range(0, len(delayed_tasks), 1000)):
    _ = dask.compute(*delayed_tasks[i:i+1000])

  0%|          | 0/1 [00:00<?, ?it/s]

In [54]:
## Load the state geojson
gdf_labels_webm=gdf_labels.to_crs(potential_image_gdf.crs)
gdf_labels_webm.reset_index(inplace=True, drop=True)
print(gdf_labels_webm.crs)
print("Number of labels:", len(gdf_labels_webm))

gdf_labels_webm.head(2)

PROJCS["WGS 84 / Pseudo-Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Mercator_1SP"],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs"],AUTHORITY["EPSG","3857"]]
Number of labels: 2079


,class_name,confidence,max_lon,min_lon,max_lat,min_lat,center_lat,center_lon,width_of_box,height_of_box,...,image_center_x,image_center_y,zoom,resolution,source,task_name,geo_box,type,geometry,style
0,Zigzag,0.933886,76.797074,76.795983,28.563398,28.562967,28.563183,76.796528,106.745525,47.777206,...,28.56,76.8,17.0,None,Drawn|Azure Maps Satellite,,None,None,"POLYGON ((8548889.723 3320190.979, 8548890.502...",{'color': 'green'}
1,Zigzag,0.922288,75.599003,75.597782,29.332909,29.332447,29.332678,75.598392,118.571004,51.207369,...,29.33,75.6,17.0,None,Drawn|Azure Maps Satellite,,None,None,"POLYGON ((8415642.296 3418027.518, 8415642.518...",{'color': 'green'}


In [55]:
images_with_label=gpd.sjoin(images_within_shape,gdf_labels_webm,predicate="contains")
images_with_label['geometry_right'] = images_with_label['index_right'].apply(lambda x: gdf_labels_webm.loc[x, 'geometry'])
print(f"Number of labels to write: {len(images_with_label)}")

print(f"Number of unique images: {len(images_with_label.drop_duplicates(subset='geometry'))}")
print(f"Number of unique labels: {len(images_with_label.drop_duplicates(subset='geometry_right'))}")

Number of labels to write: 2316
Number of unique images: 1139
Number of unique labels: 2015


In [56]:
# images_within_shape
# gdf_labels_webm

In [57]:
class_mapping = {"CFCBK": 0, "FCBK": 1, "Zigzag": 2}

def get_yolo_label(x):
    min_x, min_y, max_x, max_y = x['geometry'].bounds
    coords = np.array(x['geometry_right'].__geo_interface__['coordinates'][0])
    coords = coords[:-1]
    # normalize
    coords[:, 0] = (coords[:, 0] - min_x) / (max_x - min_x)
    coords[:, 1] = 1 - (coords[:, 1] - min_y) / (max_y - min_y)
    
    coords = coords.ravel()
    assert len(coords) == 8
    
    class_id = class_mapping[x['class_name']]
    label = np.zeros(9) * np.nan
    label[0] = class_id
    label[1:] = coords
    return label



In [58]:
images_with_label['yolo_label'] = images_with_label.apply(get_yolo_label, axis=1)
images_with_label.head(2)
ready_to_save_gdf = images_with_label.groupby("geometry").agg({"yolo_label": np.vstack, "x_idx": "first", "y_idx": "first"}).reset_index()
len(ready_to_save_gdf)

1139

In [59]:
ready_to_save_gdf['x'] = ready_to_save_gdf['x_idx'].apply(lambda x: str(int(x_values[x])))
ready_to_save_gdf['y'] = ready_to_save_gdf['y_idx'].apply(lambda x: str(int(y_values[x])))
len(ready_to_save_gdf)
# display(ready_to_save_gdf.tail(2))

1139

In [60]:
def save_label(x):
    label = x['yolo_label']
    save_path = join(save_dir, "labels", f"{x['x']}_{x['y']}.txt")
    np.savetxt(save_path, label, fmt="%d %f %f %f %f %f %f %f %f")
    
print(f"{save_dir=}")
os.makedirs(join(save_dir, "labels"), exist_ok=True)
_ = ready_to_save_gdf.apply(save_label, axis=1)

save_dir='../processed_data/haryana/'
